In [1]:
'''[![Open in Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)]
(https://studiolab.sagemaker.aws/import/github/schumadi/nd025-disaster-response/blob/master/notebooks/ML Pipeline Preparation.ipynb)'''

'[![Open in Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)]\n(https://studiolab.sagemaker.aws/import/github/schumadi/nd025-disaster-response/blob/master/notebooks/ML%20Pipeline%20Preparation.ipynb)'

# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

import pandas as pd
import numpy as np
import re

from sqlalchemy import create_engine

from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk;nltk.download('popular');nltk.download('stopwords')

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /home/studio-lab-
[nltk_data]    |     user/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /home/studio-lab-
[nltk_data]    |     user/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /home/studio-lab-
[nltk_data]    |     user/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /home/studio-lab-
[nltk_data]    |     user/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /home/studio-lab-
[nltk_data]    |     user/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to /home/studio-
[nltk_data]    |     lab-user/nltk_data.

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/disaster.db')
df = pd.read_sql_table('Messages', engine)

X = df['message']
y = df.drop(columns=['message'])

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # prep nltk transformation objects
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize and remove stop words
    lemmed = [lemmatizer.lemmatize(word, pos='v') for word in tokens if word not in stop_words]
    
    # Reduce words to their stems
    #stemmed = [stemmer.stem(word) for word in lemmed]

    return lemmed #stemmed

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
                        ('vect', CountVectorizer(tokenizer = tokenize)),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
                    ])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fb78ba770d0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(random_state=42)))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_hat = pipeline.predict(X_test)

In [9]:
for col_number, col_name in enumerate(y_test.columns):
        #returning dictionary from classification report
        print('Category: ', col_name)
        print(classification_report (y_true = y_test.iloc[:,col_number], y_pred = y_hat[:,col_number]))

Category:  related
              precision    recall  f1-score   support

           0       0.46      0.06      0.11      1210
           1       0.77      0.98      0.86      3998

    accuracy                           0.77      5208
   macro avg       0.62      0.52      0.49      5208
weighted avg       0.70      0.77      0.69      5208

Category:  request
              precision    recall  f1-score   support

           0       0.84      0.98      0.90      4318
           1       0.43      0.08      0.13       890

    accuracy                           0.83      5208
   macro avg       0.64      0.53      0.52      5208
weighted avg       0.77      0.83      0.77      5208

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5182
           1       0.00      0.00      0.00        26

    accuracy                           0.99      5208
   macro avg       0.50      0.50      0.50      5208
weighted avg       

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# define the grid of values to search
parameters = dict()
# clf__estimator__ prefix to address the model in the pipeline above
parameters['clf__estimator__n_estimators'] = [50, 100, 200]
parameters['clf__estimator__learning_rate'] = [0.01, 0.1, 1.0]

# define the evaluation procedure
cv = RepeatedKFold(n_splits=3, n_repeats=3, random_state=42)

# define the grid search procedure
grid_search = RandomizedSearchCV(verbose=2, estimator=pipeline, param_distributions=parameters, n_jobs=1, cv=3, scoring='accuracy')

# execute the grid search
grid_result = grid_search.fit(X_train, y_train)

# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END clf__estimator__learning_rate=0.01, clf__estimator__n_estimators=50; total time= 1.0min


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.